In [4]:
import torch
import numpy as np
import cv2
import time


In [5]:
#install YOLOv5
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [2]:
#Don't run to avoid dependency issues
!cd yolov5 & pip install -r requirements.txt

  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.26.4 which is incompatible.


In [6]:
# Load the YOLOv5 model
combined_model = torch.hub.load('ultralytics/yolov5', 'custom', path = r'C:\Users\krish\Downloads\2 Categories Yolov5.pt', force_reload=True)
combined_model.conf = 0.50  # Confidence threshold
combined_model.classes = [0, 1]  # Class 0 for gauze, 1 for hand

# Initialize variables for both cameras
onscreenIn, onscreenOut, countIn, countOut, frameCountIn, frameCountOut, startTime, endTime, countPlay = 0, 0, 0, 0, 0, 0, time.time(), 0, 0
condition = 7
isPaused = False

# Initialize video capture for two cameras
cap1 = cv2.VideoCapture(0)  # First camera
cap2 = cv2.VideoCapture(1)  # Second camera

while True:
    # Read frames from both cameras
    ret, frame1 = cap1.read()
    ret1, frame2 = cap2.read()

    # Check if frames are successfully captured
    if not ret or not ret1:
        print("Failed to capture frames from cameras")
        break

    # Concatenate frames horizontally for detection
    combined_frame = np.hstack((frame1, frame2))

    # Make detections
    results = combined_model(combined_frame)

    a, b, handDetected = 0, 0, False  # Initialize counters for gauze detections and hand detection flag

    for detection in results.xyxy[0]:
        # Check the class of the detected object
        detected_class = detection[5].item()

        # Gauze detection (class 0)
        if detected_class == 0:
            # Determine the camera where the gauze is detected
            if detection[0].item() < frame1.shape[1]:  # Gauze in frame 1
                a += 1
            else:
                b += 1  # Gauze in frame 2

        # Hand detection (class 1)
        elif detected_class == 1:
            handDetected = True

    # Render the detections on the image
    image = np.squeeze(results.render())

    # Update state based on hand detection
    if handDetected:
        isPaused = True
    else:
        isPaused = False

    # Process counters only if not paused
    if not isPaused:
        # Update Onscreen and Total counts for camera in
        if a != onscreenIn:
            if frameCountIn < condition:
                frameCountIn += 1
            if frameCountIn == condition:
                countIn += a - onscreenIn
                onscreenIn = a
                frameCountIn = 0
        else:
            frameCountIn = 0

        # Update Onscreen and Total counts for camera out
        if b != onscreenOut:
            if frameCountOut < condition:
                frameCountOut += 1
            if frameCountOut == condition:
                countOut += b - onscreenOut
                onscreenOut = b
                frameCountOut = 0
        else:
            frameCountOut = 0

        # Update In-Play Counter
        countPlay = countIn - onscreenIn - countOut

    # FPS Counter
    endTime = time.time()
    fps = 1 / (endTime - startTime)
    startTime = endTime

    # Add text overlays for counts and state
    # (Text positioning might need adjustment based on your frame size)
    cv2.putText(image, f'On Screen In = {onscreenIn}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(image, f'Total In = {countIn}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(image, f'On Screen Out = {onscreenOut}', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(image, f'Total Out = {countOut}', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(image, f'In Play = {countPlay}', (10, 190), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(image, f'FPS = {round(fps, 1)}', (10, 230), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    if isPaused:
        cv2.putText(image, 'System Paused - Hand Detected', (10, 270), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow('Gauze Detection', image)

    # Exit condition
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap1.release()
cap2.release()
cv2.destroyAllWindows()

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\krish/.cache\torch\hub\master.zip
YOLOv5  2024-2-6 Python-3.9.12 torch-2.1.1+cu118 CUDA:0 (NVIDIA GeForce MX250, 2048MiB)



Exception: cannot instantiate 'PosixPath' on your system. Cache may be out of date, try `force_reload=True` or see https://docs.ultralytics.com/yolov5/tutorials/pytorch_hub_model_loading for help.